# 🎬 ViarteIA - Generador de Video AI (T4 Optimized)

**Stack:**
- **T2V:** ModelScope (damo-vilab)
- **I2V:** Stable Video Diffusion (SVD-XT)
- **LLM:** Llama 3 8B (vía Groq)
- **Upscale:** Real-ESRGAN + InsightFace

> 🚀 **Instrucciones:** Ejecuta las celdas en orden. Asegúrate de tener tus claves de Groq y Ngrok.

In [ ]:
#@title 1. Instalar Dependencias y Configurar Entorno
#@markdown Esto puede tardar unos 3-5 minutos.

print("⏳ Instalando sistema base...")
!apt-get update -qq && apt-get install -y ffmpeg aria2 > /dev/null

print("📥 Configurando Node.js 20...")
!curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash - > /dev/null
!sudo apt-get install -y nodejs > /dev/null

print("🐍 Instalando librerías de Python (IA & Media)...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q diffusers transformers accelerate safetensors groq fastapi uvicorn python-multipart python-dotenv pyngrok opencv-python-headless moviepy
!pip install -q insightface onnxruntime-gpu
!pip install -q realesrgan basicsr

# Fix conocido para BasicSR en Colab
!sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' /usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py

print("✅ Entorno listo.")

In [ ]:
#@title 2. Clonar/Actualizar ViarteIA
import os

REPO_URL = "https://github.com/Juanpalojime/ViarteIA.git"
PROJECT_DIR = "/content/ViarteIA"

if os.path.exists(PROJECT_DIR):
    print("🔄 Actualizando repositorio...")
    %cd {PROJECT_DIR}
    !git reset --hard
    !git pull origin main
else:
    print("⬇️ Clonando repositorio...")
    !git clone {REPO_URL} {PROJECT_DIR}
    %cd {PROJECT_DIR}

print("📦 Instalando dependencias del Backend Node...")
%cd {PROJECT_DIR}/backend/node-api
!npm install --legacy-peer-deps > /dev/null
print("✅ Repositorio actualizado.")

In [ ]:
#@title 3. Configuración de Secretos
GROQ_API_KEY = "" #@param {type:"string"}
NGROK_AUTH_TOKEN = "" #@param {type:"string"}
JWT_SECRET = "secret_super_secure_colab_key_12345" #@param {type:"string"}

if not GROQ_API_KEY or not NGROK_AUTH_TOKEN:
    print("⚠️ ADVERTENCIA: Faltan claves importantes.")

import os

# Escribir .env para Node.js
node_env = f"""
PORT=3001
DATABASE_URL=file:./dev.db
GROQ_API_KEY={GROQ_API_KEY}
JWT_SECRET={JWT_SECRET}
PYTHON_API_URL=http://localhost:8000
NGROK_AUTH_TOKEN={NGROK_AUTH_TOKEN}
"""
with open('/content/ViarteIA/backend/node-api/.env', 'w') as f:
    f.write(node_env)

# Escribir variables para Python (usadas por os.getenv)
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["NGROK_AUTH_TOKEN"] = NGROK_AUTH_TOKEN

print("🔒 Secretos configurados.")

In [ ]:
#@title 4. Iniciar Servidores (FastAPI + Node + Ngrok)
import subprocess
import time
from pyngrok import ngrok

# 1. Configurar Ngrok
ngrok.set_auth_token(os.environ["NGROK_AUTH_TOKEN"])
public_url = ngrok.connect(3001).public_url
print(f"🚀 \n>>> FRONTEND CONNECT URL: {public_url} <<<\n")

# 2. Iniciar Backend Python (FastAPI)
print("🧠 Iniciando Motor de IA (Python)...")
python_process = subprocess.Popen(
    ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000"], 
    cwd="/content/ViarteIA/backend/python-ai/src"
)

# Esperar a que Python arranque un poco
time.sleep(5)

# 3. Iniciar Backend Node.js
print("🌐 Iniciando API Gateway (Node.js)...")
%cd /content/ViarteIA/backend/node-api
!npm run dev